In [11]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# The URL of the page you want to scrape
url = 'https://www.baseball-reference.com/teams/HOU/2023-schedule-scores.shtml'

# Fetching the HTML content
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

table = soup.find('table', id='team_schedule')
rows = table.find_all('tr')

# Extracting header columns
header = [th.get_text(strip=True) for th in rows[0].find_all('th')]

data = []
for row in rows[1:]:
    rowData = [cell.get_text(strip=True) for cell in row.find_all(['td', 'th'])]  # Get both td and th cells
    
    # Check if the first cell in the rowData is numeric
    # If not, we skip adding this row to our data
    if not rowData[0].isdigit():
        continue
    
    if len(rowData) != len(header):  # Check for discrepancies
        print(f"Row discrepancy: Expected {len(header)} columns but got {len(rowData)}.")
        rowData.extend([''] * (len(header) - len(rowData)))  # Add empty strings for missing columns
        
    data.append(rowData)

# Construct the DataFrame
df = pd.DataFrame(data, columns=header)

# Drop columns by index position
df = df.drop(df.columns[[2, 21]], axis=1)

# To save this data to CSV
#df.to_csv('astros_schedule_2023.csv', index=False)

df

,Gm#,Date,Tm,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,cLI,Streak
0,1,"Thursday, Mar 30",HOU,CHW,L,2,3,,0-1,4,1.0,Graveman,Pressly,López,2:38,N,"43,032",.92,-
1,2,"Friday, Mar 31",HOU,CHW,W,6,3,,1-1,3,0.5,Martinez,Graveman,Montero,2:40,N,"41,453",.92,+
2,3,"Saturday, Apr 1",HOU,CHW,W,6,4,,2-1,2,0.5,Stanek,Kelly,Neris,3:11,D,"37,519",.93,++
3,4,"Sunday, Apr 2",HOU,CHW,L,3,6,,2-2,3,1.5,Clevinger,Garcia,,3:06,D,"42,835",1.00,-
4,5,"Monday, Apr 3",HOU,DET,L,6,7,11,2-3,3,1.5,Wingenter,Neris,Hill,3:21,N,"29,272",.93,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,158,"Tuesday, Sep 26",HOU,SEA,L,2,6,,86-72,2,2.5,Kirby,Javier,,3:06,N,"40,035",5.15,-
158,159,"Wednesday, Sep 27",HOU,SEA,W,8,3,,87-72,2,2.5,Graveman,Miller,,3:15,N,"38,019",6.13,+
159,160,"Friday, Sep 29",HOU,ARI,W,2,1,,88-72,2,1.0,Urquidy,Gallen,Pressly,2:30,N,"36,133",4.80,++
160,161,"Saturday, Sep 30",HOU,ARI,W,1,0,,89-72,2,1.0,Verlander,Kelly,Abreu,2:51,N,"36,789",8.25,+++


In [12]:
# Change variable names

from bs4 import BeautifulSoup
import requests
import pandas as pd

def scrape_baseball_schedule(url):
    # Fetching the HTML content
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    table = soup.find('table', id='team_schedule')
    rows = table.find_all('tr')

    # Extracting header columns
    header = [th.get_text(strip=True) for th in rows[0].find_all('th')]

    games = []
    for row in rows[1:]:
        row_data = [cell.get_text(strip=True) for cell in row.find_all(['td', 'th'])]  # Get both td and th cells
        
        # Check if the first cell in the rowData is numeric
        # If not, we skip adding this row to our data
        if not row_data[0].isdigit():
            continue
        
        if len(row_data) != len(header):  # Check for discrepancies
            print(f"Row discrepancy: Expected {len(header)} columns but got {len(rowData)}.")
            row_data.extend([''] * (len(header) - len(row_data)))  # Add empty strings for missing columns
            
        games.append(row_data)

    # Construct the DataFrame
    df = pd.DataFrame(games, columns=header)

    # Drop columns by index position
    df = df.drop(df.columns[[2, 21]], axis=1)
    
    return df

In [13]:
# Define the base URL, leaving a placeholder for the year
base_url = 'https://www.baseball-reference.com/teams/HOU/{}-schedule-scores.shtml'

# Define the range of years you want to scrape
years = range(2023, 2023-7, -1)  # This will create a range from 2023 to 2017

# Loop over the years, scrape the data, and collect the DataFrames
season_dfs = []
for year in years:
    url = base_url.format(year)
    df = scrape_baseball_schedule(url)
    df['Season'] = year  # Add a column for the year/season
    season_dfs.append(df)

# Concatenate all the dataframes
astros_last_seven = pd.concat(season_dfs, ignore_index=True)
astros_last_seven


,Gm#,Date,Tm,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,cLI,Streak,Season
0,1,"Thursday, Mar 30",HOU,CHW,L,2,3,,0-1,4,1.0,Graveman,Pressly,López,2:38,N,"43,032",.92,-,2023
1,2,"Friday, Mar 31",HOU,CHW,W,6,3,,1-1,3,0.5,Martinez,Graveman,Montero,2:40,N,"41,453",.92,+,2023
2,3,"Saturday, Apr 1",HOU,CHW,W,6,4,,2-1,2,0.5,Stanek,Kelly,Neris,3:11,D,"37,519",.93,++,2023
3,4,"Sunday, Apr 2",HOU,CHW,L,3,6,,2-2,3,1.5,Clevinger,Garcia,,3:06,D,"42,835",1.00,-,2023
4,5,"Monday, Apr 3",HOU,DET,L,6,7,11,2-3,3,1.5,Wingenter,Neris,Hill,3:21,N,"29,272",.93,--,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027,158,"Wednesday, Sep 27",HOU,TEX,W,12,2,,98-60,1,up20.0,Verlander,Martinez,,3:17,D,"26,053",.18,+++,2017
1028,159,"Thursday, Sep 28",HOU,BOS,W,12,2,,99-60,1,up21.0,Peacock,Rodriguez,,3:31,N,"34,222",.10,++++,2017
1029,160,"Friday, Sep 29",HOU,BOS,W,3,2,,100-60,1,up21.0,Morton,Fister,Giles,2:47,N,"36,623",.07,+++++,2017
1030,161,"Saturday, Sep 30",HOU,BOS,L,3,6,,100-61,1,up21.0,Pomeranz,McCullers,,3:19,D,"35,722",.14,-,2017


In [22]:
astros_last_seven.to_csv('astros.csv', index=False)

In [20]:
astros = astros_last_seven

# Ensure all values in 'Attendance' are strings
astros['Attendance'] = astros['Attendance'].astype(str)

# Remove the commas from the 'Attendance' column
astros['Attendance'] = astros['Attendance'].str.replace(',', '')

# Replace empty strings with NaN (using numpy's nan)
astros['Attendance'] = astros['Attendance'].replace('', pd.NA)

# Convert the 'Attendance' column to numeric, turning non-numeric values into NaN
astros['Attendance'] = pd.to_numeric(astros['Attendance'], errors='coerce').astype('Int64')

# Display the modified DataFrame to ensure the changes have been made
astros.head




<bound method NDFrame.head of       Gm#               Date   Tm  Opp W/L   R RA Inn     W-L Rank      GB  \
0       1   Thursday, Mar 30  HOU  CHW   L   2  3         0-1    4     1.0   
1       2     Friday, Mar 31  HOU  CHW   W   6  3         1-1    3     0.5   
2       3    Saturday, Apr 1  HOU  CHW   W   6  4         2-1    2     0.5   
3       4      Sunday, Apr 2  HOU  CHW   L   3  6         2-2    3     1.5   
4       5      Monday, Apr 3  HOU  DET   L   6  7  11     2-3    3     1.5   
...   ...                ...  ...  ...  ..  .. ..  ..     ...  ...     ...   
1027  158  Wednesday, Sep 27  HOU  TEX   W  12  2       98-60    1  up20.0   
1028  159   Thursday, Sep 28  HOU  BOS   W  12  2       99-60    1  up21.0   
1029  160     Friday, Sep 29  HOU  BOS   W   3  2      100-60    1  up21.0   
1030  161   Saturday, Sep 30  HOU  BOS   L   3  6      100-61    1  up21.0   
1031  162      Sunday, Oct 1  HOU  BOS   W   4  3      101-61    1  up21.0   

            Win       Loss      S